In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from csv_parser import CSVParser
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as ts
import os

2024-10-08 16:16:26.430318: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 16:16:26.431085: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 16:16:26.435011: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-08 16:16:26.445371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 16:16:26.462868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [3]:
parser = CSVParser(folderpath="../../Project materials(1)/")
df = parser.retrieve_training_data()
df.head()

#Column 6 is lat.
#Column 7 is long.


,time,cog,sog,rot,heading,navstat,latitude,longitude,vesselId,portId,etaParsed
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,2024-01-09 23:00:00
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,2024-12-29 20:00:00
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,2024-01-02 09:00:00
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,2024-12-31 20:00:00
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,2024-01-25 12:00:00


In [4]:
def sequencing(df, window_size):
  ids = df.vesselId.unique()
  X = []
  y = []
  for j in range(len(ids)):
    df_id = df[df["vesselId"] == ids[j]].copy()
    if df_id.shape[0]>window_size:
      df_id_np = df_id.to_numpy()
      for i in range(len(df_id_np)-window_size):
        row = [r for r in df_id_np[i:i+window_size]]
        X.append(row)
        label = [df_id_np[i+window_size][6], df_id_np[i+window_size][7]]
        y.append(label)
  
  y = np.array(y)
  X = np.array(X)
  y_reshaped = y[:,:, np.newaxis]
  XandY=np.concatenate((X, np.tile(y_reshaped, (1, 1, X.shape[2]))), axis=1)
  XandY[XandY[:,0,0].argsort()]
  X = XandY[:,:-2,:]
  y = XandY[:,-2:,0]
  
  return X,y

Xs,ys = sequencing(df, 7)


[[-35.08926 -55.50466]
 [-35.09485 -55.35699]
 [-35.10507 -55.25475]
 ...
 [37.15745 144.85164]
 [37.20409 144.94732]
 [37.22231 144.98452]]


In [11]:
def convert_timestamp_to_seconds(timestamp):
    """
    Input:
    timestamp: A pandas Timestamp object
    
    Output:
    seconds: The number of seconds since the (January 1, 2023)
    """
    if isinstance(timestamp, pd.Timestamp):
        return float((timestamp - pd.Timestamp('2023-01-01')).total_seconds()) 
    elif type(timestamp)=='NaTType':
        print(timestamp)
    return float(timestamp)





Xs[:, :, 0] = np.array([np.array([convert_timestamp_to_seconds(x) for x in column]) for column in Xs[:, :, 0]])

Xs[:, :, -1] = np.array([np.array([convert_timestamp_to_seconds(x) for x in column]) for column in Xs[:, :, -1]])
print(Xs[:, :, 0])


TypeError: float() argument must be a string or a real number, not 'NaTType'

In [ ]:
train_X = Xs[:int(np.round(Xs.shape[0]*0.7))]
train_Y = ys[:int(np.round(Xs.shape[0]*0.7))]
val_X = Xs[int(np.round(Xs.shape[0]*0.7)):int(np.round(Xs.shape[0]*0.85))]
val_Y = ys[int(np.round(Xs.shape[0]*0.7)):int(np.round(Xs.shape[0]*0.85))]
test_X = Xs[int(np.round(Xs.shape[0]*0.85)):]
test_Y = ys[int(np.round(Xs.shape[0]*0.85)):]

In [5]:
print(Xs[0,:,:])
def preprocess_design(X, lo_mean, lo_std, la_mean, la_std ):
  X[:, :, 6] = (X[:, :, 6] - la_mean) / la_std
  X[:, :, 7] = (X[:, :, 7] - lo_mean) / lo_std

  return X

def preprocess_output(y,lo_mean, lo_std, la_mean, la_std):
  y[:, 0] = (y[:, 0] - la_mean) / la_std
  y[:, 1] = (y[:, 1] - lo_mean) / lo_std

  return y

[[Timestamp('2024-01-01 00:00:25') 284.0 0.7 0 88 0 -34.7437 -57.8513
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:09:08') 92.8 14.2 0 90 0 -35.16787 -56.7721
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:38:19') 90.5 14.3 0 88 0 -35.16863 -56.63185
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:58:55') 88.2 14.3 0 86 0 -35.16805 -56.5319
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 07:15:56') 88.3 12.3 0 86 0 -35.16715 -56.45306
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 07:28:15') 86.8 11.7 0 85 0 -35.16646 -56.40306
  '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01

In [8]:
lang_t_mean = np.mean(train_X[:, :, 6])
lang_t_std = np.std(train_X[:, :, 6])

long_t_mean = np.mean(train_X[:, :, 7])
long_t_std = np.std(train_X[:, :, 7])

train_X = preprocess_design(train_X, long_t_mean, long_t_std, lang_t_mean, lang_t_std)
train_Y = preprocess_output(train_Y, long_t_mean, long_t_std, lang_t_mean, lang_t_std)
val_X = preprocess_design(val_X, long_t_mean, long_t_std, lang_t_mean, lang_t_std)
val_Y = preprocess_output(val_Y, long_t_mean, long_t_std, lang_t_mean, lang_t_std)
test_X = preprocess_design(test_X, long_t_mean, long_t_std, lang_t_mean, lang_t_std)
test_Y = preprocess_output(test_Y, long_t_mean, long_t_std, lang_t_mean, lang_t_std)

In [32]:
def convert_timestamp_to_seconds(timestamp):
    """
    Input:
    timestamp: A pandas Timestamp object
    
    Output:
    seconds: The number of seconds since the (January 1, 2023)
    """
    if isinstance(timestamp, pd.Timestamp):
        return float((timestamp - pd.Timestamp('2023-01-01')).total_seconds())
    return float(timestamp)



[[Timestamp('2024-01-01 00:00:25') 284.0 0.7 0 88 0 -3.61289665639214
  -1.1814709265047045 '61e9f3a8b937134a3c4bfdf7'
  '61d371c43aeaecc07011a37f' Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:09:08') 92.8 14.2 0 90 0 -3.633415549939463
  -1.1625419351294957 '61e9f3a8b937134a3c4bfdf7'
  '61d371c43aeaecc07011a37f' Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:38:19') 90.5 14.3 0 88 0 -3.6334523143478274
  -1.16008197308319 '61e9f3a8b937134a3c4bfdf7' '61d371c43aeaecc07011a37f'
  Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 06:58:55') 88.2 14.3 0 86 0 -3.633424257299339
  -1.1583288664412772 '61e9f3a8b937134a3c4bfdf7'
  '61d371c43aeaecc07011a37f' Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 07:15:56') 88.3 12.3 0 86 0 -3.63338072049996
  -1.156946025744445 '61e9f3a8b937134a3c4bfdf7'
  '61d371c43aeaecc07011a37f' Timestamp('2024-01-09 23:00:00')]
 [Timestamp('2024-01-01 07:28:15') 86.8 11.7 0 85 0 -3.633347342287103
  -1.1560690339275

In [9]:
model1 = Sequential()
model1.add(InputLayer((7, 11))) 
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(2, 'linear'))

model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        19,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,994 (78.10 KB)

 Trainable params: 19,994 (78.10 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
cp5 = ModelCheckpoint('model5.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [11]:
model1.fit(train_X, train_Y, validation_data=(val_X, val_Y), epochs=10, callbacks=[cp5])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).